<a href="https://colab.research.google.com/github/bidentity/Capstone_movilens/blob/master/IBM_Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A JAPANESE IN BARCELONA¶

## 1. Introduction



Barcelona is one of the cities in the world with more cosmopolitanism in Europe. Barcelona has one of the largest catering offers per square meter in Europe.

Not all restaurants from same category in barcelona are the same, they do not offer the same service. This makes it sometimes difficult to discern which neighborhoods are the most suitable for our needs depending on their service delivery.

In this project we will analyze which Neighborhood of barcelona has most Japanese restaurants.


## 2. Data and Source

For the purpose of this study, we will asume that we do not have any dataset by our own. In fact we are going to "steal" data from the web.  For our objective, we are going to scrap Barcelona's neighbourhood data from **Wikpedia** : https://es.wikipedia.org/wiki/Categor%C3%ADa:Barrios_de_Barcelona.

* After that we will positionate geolocation (latitude , and longitude ) to each  neighborhood, and overlaping the coordinates on a geographic map.

* Later on, geographic data will be used on  a **Foursquare api** will be used in order to search venues in each neighborhood.

* Finally we will select only "japanese Restaurants" and group by neighborhood in order to analyse them.

## 3. Methodology

In [3]:
!pip install -q geocoder

     |████████████████████████████████| 102kB 3.5MB/s 


In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [0]:
# send the GET request
data = requests.get("https://es.wikipedia.org/wiki/Categor%C3%ADa:Barrios_de_Barcelona").text

In [0]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [0]:
# create a list to store neighborhood data
neighborhoodList = []

In [0]:
# append the data into the list
for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [12]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df

# from 0 to 11 is information that we do not need  for the purpose of this project. SO we are going to delete them and to maintain rows from 12 to the end.



,Neighborhood
0,v
1,d
2,e
3,► Antiguos barrios de Barcelona‎ (7 págs.)
4,► La Barceloneta‎ (8 págs.)
5,► Distrito 22@‎ (10 págs.)
6,"► El Raval‎ (1 cat, 9 págs.)"
7,"► Barrio Gótico‎ (3 cat, 31 págs.)"
8,► La Maternidad y San Ramón‎ (7 págs.)
9,"► Sant Pere, Santa Caterina i la Ribera‎ (2 c..."


In [13]:
kl_df2 = kl_df.drop(kl_df.index[[0,1,2,3,4,5,6,7,8,9,10,11 ]])
kl_df2

,Neighborhood
12,La Barceloneta
13,Barón de Viver
14,El Buen Pastor (Barcelona)
15,La Bordeta (Barcelona)
16,Camp d'en Grassot i Gràcia Nova
17,Can Baró
18,Can Peguera
19,Canyelles (barrio)
20,Ciudad Meridiana
21,El Clot


In [14]:
kl_df2.shape

(76, 1)

In [0]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Barcelona, Spain'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
# merge the coordinates into the original dataframe
kl_df2['Latitude'] = df_coords['Latitude']
kl_df2['Longitude'] = df_coords['Longitude']

In [61]:
print(kl_df2.shape)
kl_df2

(76, 3)


,Neighborhood,Latitude,Longitude
12,La Barceloneta,41.381850,2.191510
13,Barón de Viver,41.448040,2.199630
14,El Buen Pastor (Barcelona),41.389100,2.121520
15,La Bordeta (Barcelona),41.369860,2.136220
16,Camp d'en Grassot i Gràcia Nova,41.408590,2.163820
17,Can Baró,41.416650,2.162530
18,Can Peguera,41.435080,2.166530
19,Canyelles (barrio),41.271134,1.728693
20,Ciudad Meridiana,41.460320,2.175730
21,El Clot,41.409120,2.192080


In [0]:


# save the DataFrame as CSV file
kl_df2.to_csv("kl_df2.csv", index=False)



 Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [63]:
# get the coordinates of Kuala Lumpur
address = 'Barcelona, Spain'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Spain {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Spain 41.3828939, 2.1774322.


In [64]:
# create map of Barcelona using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df2['Latitude'], kl_df2['Longitude'], kl_df2['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [0]:


# save the map as HTML file
map_kl.save('map_kl.html')



* Use the Foursquare API to explore the neighborhoods

In [27]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


In [0]:
radius = 2000
LIMIT = 100
CLIENT_ID = 'X05YNJIZO4FRI42P2MNDWQOUMIAZJV4Z0SWA4LZQGKOKBGA4'
CLIENT_SECRET = 'KDR1IOSQ2J5KVHVTVWIBXRWBKB5ZJILPGV5NJI03NZCRSQI0'

venues = []

for lat, long, neighborhood in zip(kl_df2['Latitude'], kl_df2['Longitude'], kl_df2['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6805, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,La Barceloneta,41.38185,2.19151,Baluard Barceloneta,41.380047,2.189250,Bakery
1,La Barceloneta,41.38185,2.19151,BRO,41.380214,2.189007,Burger Joint
2,La Barceloneta,41.38185,2.19151,Platja de la Barceloneta (Playa de la Barcelon...,41.379610,2.193038,Beach
3,La Barceloneta,41.38185,2.19151,Somorrostro,41.379156,2.189100,Spanish Restaurant
4,La Barceloneta,41.38185,2.19151,La Cova Fumada,41.379254,2.189254,Tapas Restaurant


In [32]:


print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))



There are 245 uniques categories.


In [34]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:245]

array(['Bakery', 'Burger Joint', 'Beach', 'Spanish Restaurant',
       'Tapas Restaurant', 'Plaza', 'Wine Shop',
       'Mediterranean Restaurant', 'Beer Bar', 'Pizza Place',
       'Paella Restaurant', 'Salon / Barbershop', 'Restaurant', 'Lounge',
       'Sushi Restaurant', 'Cocktail Bar', 'Mexican Restaurant',
       'Gastropub', 'Argentinian Restaurant', 'Hawaiian Restaurant',
       'Circus', 'Arcade', 'Coffee Shop', 'Japanese Restaurant',
       'Hotel Bar', 'Park', 'Ice Cream Shop', 'Brazilian Restaurant',
       'Steakhouse', 'Spa', 'Market', 'Wine Bar', 'Dessert Shop',
       'Food & Drink Shop', 'Fountain', 'Camera Store', 'Bar', 'Café',
       'Hotel', 'Seafood Restaurant', 'Board Shop', 'Hotel Pool',
       'Latin American Restaurant', 'Neighborhood', 'Italian Restaurant',
       'Gym / Fitness Center', 'Ramen Restaurant', 'Pier',
       'Indian Restaurant', 'Molecular Gastronomy Restaurant',
       'Shopping Mall', 'Electronics Store', 'Cupcake Shop', 'Food',
       'Brewer

In [0]:
# Create a new Dataframe with venues as Japanese Rstaurant.
new_venues_df = venues_df[venues_df.VenueCategory == 'Japanese Restaurant']

In [36]:
new_venues_df

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
34,La Barceloneta,41.381850,2.191510,Restaurant Ají,41.386453,2.197158,Japanese Restaurant
151,Barón de Viver,41.448040,2.199630,Restaurante Shokudo,41.439386,2.192142,Japanese Restaurant
250,El Buen Pastor (Barcelona),41.389100,2.121520,Nomo Sarria,41.400153,2.121276,Japanese Restaurant
270,El Buen Pastor (Barcelona),41.389100,2.121520,La cuina de l'Uribou,41.386799,2.140475,Japanese Restaurant
285,El Buen Pastor (Barcelona),41.389100,2.121520,Maido,41.378450,2.137591,Japanese Restaurant
328,La Bordeta (Barcelona),41.369860,2.136220,Maido,41.378450,2.137591,Japanese Restaurant
585,Can Baró,41.416650,2.162530,Wakasa,41.403191,2.168923,Japanese Restaurant
731,El Clot,41.409120,2.192080,UN-ZA,41.405376,2.193442,Japanese Restaurant
1113,Diagonal Mar i el Front Marítim del Poblenou,41.402380,2.208590,UN-ZA,41.405376,2.193442,Japanese Restaurant
1178,La Dreta de l'Eixample,41.394980,2.167480,Ekubo Barcelona,41.383930,2.156294,Japanese Restaurant


In [48]:
by_n = new_venues_df.groupby("Neighborhood",sort=False).count()
print(by_n)

                                              Latitude  Longitude  VenueName  \
Neighborhood                                                                   
La Barceloneta                                       1          1          1   
Barón de Viver                                       1          1          1   
El Buen Pastor (Barcelona)                           3          3          3   
La Bordeta (Barcelona)                               1          1          1   
Can Baró                                             1          1          1   
El Clot                                              1          1          1   
Diagonal Mar i el Front Marítim del Poblenou         1          1          1   
La Dreta de l'Eixample                               1          1          1   
El Baix Guinardó                                     2          2          2   
El Campo del Arpa del Clot                           2          2          2   
El Coll                                 

## 5. Discussion

As we have seen throughout the analysis, it is possible to carry out an analysis without having any information. If it is true, on the other hand that the initial results, especially in the geolocation, has not responded exactly, so it would be necessary to purify the information obtained by scrap. Similarly, this geographical misalignment has not affected the final result, nor the conclusions we wanted at the beginning of the analysis.

## 6. Conclussion

As a conclusion we can say that the average range of Japanese restaurants in Barcelona is between 1 and 3, being the neighborhoods of **Sant Gervasi-Galvany ** and **Sant Gervasi-La Bonanova **, the two neighborhoods with the highest average of Japanese restaurants, being **5** the highest number